In [1]:
#!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
import pandas as pd
# Bibliotecas Auxiliares
import seaborn as sns
import numpy as np
import collections
import matplotlib.pyplot as plt
import nest_asyncio
nest_asyncio.apply()
import tensorflow_federated as tff
np.random.seed(0)
from collections import Counter

import collections
import functools
import os
import time
from scipy import stats

from sklearn.model_selection import train_test_split

import datetime;
  
import warnings
warnings.filterwarnings("ignore")

print(tf.__version__)

2023-03-30 17:37:35.515094: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 17:37:35.923458: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-30 17:37:36.887923: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/
2023-03-30 17:37:36.890094: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7';

2.10.0


In [2]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    array = []
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    results['matrix'] = np.array(matrix,dtype=object)
    results['TP'] = matrix[0][0]
    results['FP'] = matrix[0][1]
    results['FN'] = matrix[1][0]
    results['TN'] = matrix[1][1]
    
    array.append(accuracy)
    array.append(precision)
    array.append(recall)
    array.append(f1)
    array.append(kappa)
    array.append(auc)
    array.append(np.array(matrix,dtype=object))
    array.append(matrix[0][0]) # TP
    array.append(matrix[0][1]) # FP
    array.append(matrix[1][0]) # FN
    array.append(matrix[1][1]) # TN
    
    return results, array

def showGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
    return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score]
    
def create_dataset_time_series_with_one_output(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        v = X.iloc[i:(i + window_time_steps)].values
        labels = y.iloc[i: i + window_time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))

In [3]:
# configs
EPOCHS = 1
NUM_EPOCHS = EPOCHS
BATCH_SIZE = 32

#TIME_SERIES_SIZE = 4   # Determines the window size. Ex (4,9)
#TIME_STEP_SHIFT  = 1   # Determines specifies the number of steps to move the window forward at each iteration.

baseFolder = "../data_2019_processed/"

# selected features
inputFeatures = ["activity","location","day_of_week",
                 "light","phone_lock","proximity",
                 "sound","time_to_next_alarm", "minutes_day"]
# outputs
outputClasses = ["awake","asleep"]
#outputClasses = ["class"]

In [4]:
# test data comprising 25% of the data. It must be fixed to all models being evaluated
X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized.csv")

# input folder
inputFolders = baseFolder

            
dsTrain = ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
            '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
            '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
            '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
            #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
            '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
            'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
            'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
            'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
            'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
            'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
            'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', 
            'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
            'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
            'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
            'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
            'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
            'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
            'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
            'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', 
            'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
            'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
            'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
            'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4', 
            'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
            'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
            'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
            'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
            'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
            'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']


# client datasets used on the training process
dsTrain =  ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
            '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
            '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
            '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
            #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
            '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
            'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
            'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
            'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
            'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
            'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
            'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', 
            'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
            'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
            'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
            'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
            'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
            'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
            'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
            #['PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc'], 
            'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
            #['rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw'], 
            #['RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI'], 
            'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4'] 
            #['VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is'], 
            #['Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw'], 
            #['XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA'], 
            #['YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw'], 
            #['ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM'], 
            #['ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']]

In [5]:
# load cliend data
clientList = []

for i in range(0,len(dsTrain)):
    print (i," ", str(inputFolders)+"student_"+dsTrain[i]+"_numeric.csv") #_numeric
    # load client data
    dataset = pd.read_csv(inputFolders+"student_"+dsTrain[i]+"_numeric.csv")
    
    # print(dataset)
    y_train = dataset['class'].copy()
    
    # does not add datasets that dont have instances from both classes
    if y_train.sum() != 0 and (y_train.sum() != len(y_train)):
        clientList.append(dataset)
        
print("Total",(len(clientList)))

0   ../data_2019_processed/student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs_numeric.csv
1   ../data_2019_processed/student_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA_numeric.csv
2   ../data_2019_processed/student_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0_numeric.csv
3   ../data_2019_processed/student_2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys_numeric.csv
4   ../data_2019_processed/student_7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA_numeric.csv
5   ../data_2019_processed/student_a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4_numeric.csv
6   ../data_2019_processed/student_ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc_numeric.csv
7   ../data_2019_processed/student_Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U_numeric.csv
8   ../data_2019_processed/student_CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA_numeric.csv
9   ../data_2019_processed/student_DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc_numeric.csv
10   ../data_2019_processed/student_DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA_numeric.csv
11   ../

In [6]:
# undestand the dataset by looking on their infos
print(X_test.info())

X_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134888 entries, 0 to 134887
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            134888 non-null  float64
 1   location            134888 non-null  float64
 2   timestamp           134888 non-null  float64
 3   time_to_next_alarm  134888 non-null  float64
 4   sound               134888 non-null  float64
 5   proximity           134888 non-null  float64
 6   phone_lock          134888 non-null  float64
 7   light               134888 non-null  float64
 8   day_of_week         134888 non-null  float64
 9   minutes_day         134888 non-null  float64
 10  timestamp_text      134888 non-null  object 
 11  class               134888 non-null  object 
dtypes: float64(10), object(2)
memory usage: 12.3+ MB
None


,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.75,1.0,0.000552,0.000000,0.515992,1.0,0.0,0.000000,1.000000,0.678249,2018-05-14 16:16:08,awake
1,0.25,1.0,0.000552,0.000000,0.542171,0.0,1.0,0.000007,1.000000,0.678944,2018-05-14 16:17:39,awake
2,0.25,1.0,0.000552,0.000000,0.515992,0.0,1.0,0.000000,1.000000,0.679639,2018-05-14 16:18:39,awake
3,0.00,1.0,0.000552,0.000000,0.515992,0.0,1.0,0.000000,1.000000,0.680334,2018-05-14 16:19:09,awake
4,0.25,1.0,0.000552,0.000000,0.531341,0.0,1.0,0.000000,1.000000,0.681029,2018-05-14 16:20:09,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
134883,0.25,1.0,0.007015,0.000099,0.000000,1.0,1.0,0.000056,0.166667,0.510076,2018-06-13 12:14:37,awake
134884,0.25,1.0,0.007015,0.000694,0.000000,1.0,1.0,0.000078,0.166667,0.512856,2018-06-13 12:18:08,awake
134885,0.25,1.0,0.007015,0.000595,0.000000,1.0,1.0,0.000078,0.166667,0.513551,2018-06-13 12:19:08,awake
134886,0.25,1.0,0.007016,0.000595,0.000000,1.0,1.0,0.000085,0.166667,0.513551,2018-06-13 12:19:38,awake


In [7]:
# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset

# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the input dataset
for i in range(0,len(clientList)):
    clientList[i] = transform_output_numerical_class_into_one_hot_encoding(clientList[i])
    #print (clientList[i])

In [8]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134888 entries, 0 to 134887
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            134888 non-null  float64
 1   location            134888 non-null  float64
 2   timestamp           134888 non-null  float64
 3   time_to_next_alarm  134888 non-null  float64
 4   sound               134888 non-null  float64
 5   proximity           134888 non-null  float64
 6   phone_lock          134888 non-null  float64
 7   light               134888 non-null  float64
 8   day_of_week         134888 non-null  float64
 9   minutes_day         134888 non-null  float64
 10  timestamp_text      134888 non-null  object 
 11  class               134888 non-null  object 
 12  awake               134888 non-null  uint8  
 13  asleep              134888 non-null  uint8  
dtypes: float64(10), object(2), uint8(2)
memory usage: 12.6+ MB


In [9]:
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    for column in outputClasses:
        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# transforms the data
X_test = transform_data_type(X_test)

X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134888 entries, 0 to 134887
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            134888 non-null  float32
 1   location            134888 non-null  float32
 2   timestamp           134888 non-null  float64
 3   time_to_next_alarm  134888 non-null  float32
 4   sound               134888 non-null  float32
 5   proximity           134888 non-null  float32
 6   phone_lock          134888 non-null  float32
 7   light               134888 non-null  float32
 8   day_of_week         134888 non-null  float32
 9   minutes_day         134888 non-null  float32
 10  timestamp_text      134888 non-null  object 
 11  class               134888 non-null  object 
 12  awake               134888 non-null  float32
 13  asleep              134888 non-null  float32
dtypes: float32(11), float64(1), object(2)
memory usage: 8.7+ MB


In [10]:
# selects the data to train and test
X_test_data = X_test[inputFeatures]
y_test_label = X_test[outputClasses]

# transtorm data to tensor slices
client_test_dataset = tf.data.Dataset.from_tensor_slices((X_test_data.values, y_test_label.values))

#client_test_dataset = client_test_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE, drop_remainder=True)
client_test_dataset = client_test_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE)

print(client_test_dataset.element_spec)
client_test_dataset

(TensorSpec(shape=(None, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))


2023-03-29 12:36:22.312704: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-29 12:36:22.375974: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-29 12:36:22.376267: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-29 12:36:22.378986: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

<BatchDataset element_spec=(TensorSpec(shape=(None, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [11]:
federated_training_data = []
# transform the data
for i in range(0,len(clientList)):
    # selects the data to train and test
    data   = clientList[i][inputFeatures]
    labels = clientList[i][outputClasses]
    # transform the data to tensor slices
    client_train_dataset = tf.data.Dataset.from_tensor_slices((data.values, labels.values))
    # apply the configs
    client_train_dataset = client_train_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE)
    # transform the data to
    federated_training_data.append(client_train_dataset)

In [12]:
def create_keras_model():
    return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(9,)),
      #tf.keras.layers.Dense(9, activation=tf.keras.activations.relu), 
      tf.keras.layers.Dense(16, activation=tf.keras.activations.relu),
      tf.keras.layers.Dense(12, activation=tf.keras.activations.relu),
      tf.keras.layers.Dense(2, activation=tf.keras.activations.softmax)
      #tf.keras.layers.Dense(2, activation=tf.nn.sigmoid)
    ])

In [13]:
keras_model = create_keras_model()
#keras_model.summary()
keras_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                160       
                                                                 
 dense_1 (Dense)             (None, 12)                204       
                                                                 
 dense_2 (Dense)             (None, 2)                 26        
                                                                 
Total params: 390
Trainable params: 390
Non-trainable params: 0
_________________________________________________________________


In [14]:
def model_fn():
    # We _must_ create a new model here, and _not_ capture it from an external
    # scope. TFF will call this within different graph contexts.
    #keras_model = create_keras_model()
    keras_model = create_keras_model()
    return tff.learning.from_keras_model(
      keras_model,
      input_spec=client_train_dataset.element_spec,
      loss=tf.keras.losses.CategoricalCrossentropy(), #BinaryCrossentropy
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [15]:
fed_avg_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),#client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [16]:
print(fed_avg_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[9,16],
      float32[16],
      float32[16,12],
      float32[12],
      float32[12,2],
      float32[2]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64
  >
>@SERVER)


In [17]:
state = fed_avg_process.initialize()

In [18]:
roundData = []

columns = ['NN_type','units','epochs','batch_size','max_iterations','Users',
           'round_iteration','start_time','end_time','round_time_s','round_time_m',
           'class','accuracy','precision','recall','f1_score','cohen_kappa_score','roc_auc_score','confusion_matrix',
           'TP','FP','FN','TN']

MAX_ITERATIONS = 120
#NUM_EPOCHS
#BATCH_SIZE
NN_type = 'MLP'
UNITS_NUMBER = 12
USER_NUMBER = len(dsTrain)

generalData = [NN_type,UNITS_NUMBER,NUM_EPOCHS,BATCH_SIZE,MAX_ITERATIONS,USER_NUMBER]

for i in range(0,MAX_ITERATIONS):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}, metrics={}'.format(i,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data,verbose=0)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 
    
    # generate time metrics
    current_time2 = datetime.datetime.now()
    time_stamp2 = current_time2.timestamp()
    processing_time_s = (time_stamp2-time_stamp)
    # generate general metrics
    rowData = [i,current_time,current_time2,processing_time_s,(processing_time_s)/60]

    print('')
    print('awake')    
    res,resA = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
   
    #columns = ['NN_type','units','epochs','batch_size','max_iterations',''Users',
    #            round_iteration','start_time','end_time','round_time_s','round_time_m',
    #           'class','accuracy','precision','recall','f1_score','cohen_kappa_score','roc_auc_score','confusion_matrix',
    #           'TP','FP','FN','TN']
    # new data
    classData = []
    classData = np.concatenate((np.array(['awake'],dtype=object), resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    roundData.append(classData)
    
    print('')
    print('asleep')
    res,resA = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    # new data
    classData = np.concatenate((np.array(['asleep'],dtype=object), resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    roundData.append(classData)
    print('')
    print('Global')
    resA = showGlobalMetrics(test) #return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score
    # new data
    classData = np.concatenate((np.array(['avg'],dtype=object), resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    roundData.append(classData)
    print('')
    print(i,"End timestamp:", time_stamp2,current_time2)
    print('')
    print('')

0 Start timestamp: 1680089786.320882 2023-03-29 12:36:26.320882


2023-03-29 12:36:32.850820: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


round  0, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8375952), ('loss', 0.37621114), ('num_examples', 407451), ('num_batches', 12745)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

0.98506576 0.014934292 0.5
[[0.93323827 0.06676181]
 [0.81691074 0.18308927]
 [0.8184524  0.18154752]
 ...
 [0.8343657  0.16563423]
 [0.8343657  0.16563423]
 [0.78098255 0.2190174 ]]

awake
Accuracy: 0.813660
Precision: 0.813660
Recall: 1.000000
F1 score: 0.897258
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix
[[     0  25135]
 [     0 109753]]

asleep
Accuracy: 0.813660
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix
[[109753      0]
 [ 25135      0]]

Global
2
accuracy:  0.8136602218136528
precision:  0.4068301109068264
recall:  0.5
f1_score:  0.4486288071091927
c

round  7, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.88106793), ('loss', 0.2947193), ('num_examples', 407451), ('num_batches', 12745)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

0.9996352 0.00036479102 0.5
[[0.9835517  0.01644831]
 [0.8829787  0.11702134]
 [0.88907063 0.11092941]
 ...
 [0.9737127  0.02628727]
 [0.9737127  0.02628727]
 [0.9679584  0.03204165]]

awake
Accuracy: 0.813149
Precision: 0.813881
Recall: 0.998752
F1 score: 0.896889
Cohens kappa: 0.002359
ROC AUC: 0.500729
\Confusion Matrix
[[    68  25067]
 [   137 109616]]

asleep
Accuracy: 0.813149
Precision: 0.331707
Recall: 0.002705
F1 score: 0.005367
Cohens kappa: 0.002359
ROC AUC: 0.500729
\Confusion Matrix
[[109616    137]
 [ 25067     68]]

Global
2
accuracy:  0.8131486863175376
precision:  0.5727944008722922
recall:  0.5007285667191885
f1_score:  0.45

round  14, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.88564515), ('loss', 0.2852855), ('num_examples', 407451), ('num_batches', 12745)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

0.9997482 0.00025183544 0.5
[[0.9780673  0.02193271]
 [0.85040873 0.14959124]
 [0.859642   0.14035794]
 ...
 [0.97839487 0.02160515]
 [0.97839487 0.02160515]
 [0.97933674 0.02066321]]

awake
Accuracy: 0.817048
Precision: 0.818121
Recall: 0.996738
F1 score: 0.898640
Cohens kappa: 0.046108
ROC AUC: 0.514582
\Confusion Matrix
[[   815  24320]
 [   358 109395]]

asleep
Accuracy: 0.817048
Precision: 0.694800
Recall: 0.032425
F1 score: 0.061958
Cohens kappa: 0.046108
ROC AUC: 0.514582
\Confusion Matrix
[[109395    358]
 [ 24320    815]]

Global
2
accuracy:  0.8170482177806773
precision:  0.7564601443457617
recall:  0.5145815178376257
f1_score:  0.4

...

round  90, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.9153371), ('loss', 0.22617447), ('num_examples', 407451), ('num_batches', 12745)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

0.9999809 1.9019992e-05 0.5
[[0.9200536  0.07994637]
 [0.8610543  0.13894574]
 [0.8676397  0.13236026]
 ...
 [0.8897644  0.11023559]
 [0.8897644  0.11023559]
 [0.91429585 0.08570408]]

awake
Accuracy: 0.819102
Precision: 0.820472
Recall: 0.995499
F1 score: 0.899551
Cohens kappa: 0.069206
ROC AUC: 0.522178
\Confusion Matrix
[[  1228  23907]
 [   494 109259]]

asleep
Accuracy: 0.819102
Precision: 0.713124
Recall: 0.048856
F1 score: 0.091447
Cohens kappa: 0.069206
ROC AUC: 0.522178
\Confusion Matrix
[[109259    494]
 [ 23907   1228]]

Global
2
accuracy:  0.8191017733230532
precision:  0.7667982333507994
recall:  0.5221775803642753
f1_score:  0.4

round  97, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.9157764), ('loss', 0.22470152), ('num_examples', 407451), ('num_batches', 12745)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

0.9999863 1.3652394e-05 0.5
[[0.9230875  0.07691254]
 [0.8779841  0.12201588]
 [0.8848659  0.1151341 ]
 ...
 [0.87135124 0.12864882]
 [0.87135124 0.12864882]
 [0.90558404 0.09441592]]

awake
Accuracy: 0.817982
Precision: 0.818616
Recall: 0.997267
F1 score: 0.899153
Cohens kappa: 0.051208
ROC AUC: 0.516198
\Confusion Matrix
[[   883  24252]
 [   300 109453]]

asleep
Accuracy: 0.817982
Precision: 0.746407
Recall: 0.035130
F1 score: 0.067102
Cohens kappa: 0.051208
ROC AUC: 0.516198
\Confusion Matrix
[[109453    300]
 [ 24252    883]]

Global
2
accuracy:  0.8179823260779313
precision:  0.7825115238525358
recall:  0.5161984429615959
f1_score:  0.4

[[109287    466]
 [ 23766   1369]]

Global
2
accuracy:  0.8203546646106399
precision:  0.7837142483078847
recall:  0.5251099933093702
f1_score:  0.500860183718237
cohen_kappa_score:  0.07814511982259575
roc_auc_score:  0.5251099933093701

103 End timestamp: 1680097626.242627 2023-03-29 14:47:06.242627


104 Start timestamp: 1680097626.691067 2023-03-29 14:47:06.691067
round  104, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.91502047), ('loss', 0.22512402), ('num_examples', 407451), ('num_batches', 12745)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

0.9999857 1.4328853e-05 0.5
[[0.94259137 0.05740857]
 [0.90073574 0.09926423]
 [0.9096488  0.09035119]
 ...
 [0.8645247  0.13547531]
 [0.8645247  0.13547531]
 [0.9091719  0.0908281 ]]

awake
Accuracy: 0.822957
Precision: 0.824849
Recall: 0.993340
F1 score: 0.901288
Cohens kapp

[[  2186  22949]
 [   860 108893]]

asleep
Accuracy: 0.823491
Precision: 0.717663
Recall: 0.086970
F1 score: 0.155140
Cohens kappa: 0.119679
ROC AUC: 0.539567
\Confusion Matrix
[[108893    860]
 [ 22949   2186]]

Global
2
accuracy:  0.8234905996085642
precision:  0.7717990488884091
recall:  0.5395672916785562
f1_score:  0.5282953814962822
cohen_kappa_score:  0.119679059540998
roc_auc_score:  0.5395672916785562

110 End timestamp: 1680098127.136836 2023-03-29 14:55:27.136836


111 Start timestamp: 1680098127.595209 2023-03-29 14:55:27.595209
round  111, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.9159261), ('loss', 0.2237422), ('num_examples', 407451), ('num_batches', 12745)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

0.99998116 1.8783696e-05 0.5
[[0.94522446 0.05477555]
 [0.91311496 0.08688506]
 [0.9221675  0.07783256]

0.99997413 2.5909158e-05 0.5
[[0.956334   0.04366605]
 [0.9262683  0.07373173]
 [0.93746233 0.06253764]
 ...
 [0.8330406  0.16695943]
 [0.8330406  0.16695943]
 [0.86879975 0.13120024]]

awake
Accuracy: 0.824736
Precision: 0.830605
Recall: 0.985604
F1 score: 0.901490
Cohens kappa: 0.157320
ROC AUC: 0.553952
\Confusion Matrix
[[  3074  22061]
 [  1580 108173]]

asleep
Accuracy: 0.824736
Precision: 0.660507
Recall: 0.122300
F1 score: 0.206385
Cohens kappa: 0.157320
ROC AUC: 0.553952
\Confusion Matrix
[[108173   1580]
 [ 22061   3074]]

Global
2
accuracy:  0.8247360773382362
precision:  0.7455560009173003
recall:  0.5539518102071144
f1_score:  0.5539377022912227
cohen_kappa_score:  0.15732027238870927
roc_auc_score:  0.5539518102071144

117 End timestamp: 1680098630.100297 2023-03-29 15:03:50.100297


118 Start timestamp: 1680098630.551543 2023-03-29 15:03:50.551543
round  118, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accur

In [19]:
dataMetrics = pd.DataFrame(data=roundData,columns=columns) 

dataMetrics

,NN_type,units,epochs,batch_size,max_iterations,Users,round_iteration,start_time,end_time,round_time_s,...,precision,recall,f1_score,cohen_kappa_score,roc_auc_score,confusion_matrix,TP,FP,FN,TN
0,MLP,12,1,32,120,20,0,2023-03-29 12:36:26.320882,2023-03-29 12:37:49.923717,83.602835,...,0.813660,1.000000,0.897258,0.000000,0.500000,"[[0, 25135], [0, 109753]]",0.0,25135.0,0.0,109753.0
1,MLP,12,1,32,120,20,0,2023-03-29 12:36:26.320882,2023-03-29 12:37:49.923717,83.602835,...,0.000000,0.000000,0.000000,0.000000,0.500000,"[[109753, 0], [25135, 0]]",109753.0,0.0,25135.0,0.0
2,MLP,12,1,32,120,20,0,2023-03-29 12:36:26.320882,2023-03-29 12:37:49.923717,83.602835,...,0.406830,0.500000,0.448629,0.000000,0.500000,None,NaN,NaN,NaN,NaN
3,MLP,12,1,32,120,20,1,2023-03-29 12:37:50.396866,2023-03-29 12:39:05.349814,74.952948,...,0.813660,1.000000,0.897258,0.000000,0.500000,"[[0, 25135], [0, 109753]]",0.0,25135.0,0.0,109753.0
4,MLP,12,1,32,120,20,1,2023-03-29 12:37:50.396866,2023-03-29 12:39:05.349814,74.952948,...,0.000000,0.000000,0.000000,0.000000,0.500000,"[[109753, 0], [25135, 0]]",109753.0,0.0,25135.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,MLP,12,1,32,120,20,118,2023-03-29 15:03:50.551543,2023-03-29 15:05:01.145174,70.593631,...,0.649038,0.118162,0.199926,0.151206,0.551765,"[[108147, 1606], [22165, 2970]]",108147.0,1606.0,22165.0,2970.0
356,MLP,12,1,32,120,20,118,2023-03-29 15:03:50.551543,2023-03-29 15:05:01.145174,70.593631,...,0.739473,0.551765,0.550453,0.151206,0.551765,None,NaN,NaN,NaN,NaN
357,MLP,12,1,32,120,20,119,2023-03-29 15:05:01.599827,2023-03-29 15:06:13.221005,71.621178,...,0.830235,0.986406,0.901608,0.154648,0.552841,"[[2998, 22137], [1492, 108261]]",2998.0,22137.0,1492.0,108261.0
358,MLP,12,1,32,120,20,119,2023-03-29 15:05:01.599827,2023-03-29 15:06:13.221005,71.621178,...,0.667706,0.119276,0.202397,0.154648,0.552841,"[[108261, 1492], [22137, 2998]]",108261.0,1492.0,22137.0,2998.0


In [20]:
dataMetrics.to_csv("result1epochsMLPunbalanced2.csv", sep=',', encoding='utf-8', index=False)

In [21]:
metrics = ['class',"epochs","accuracy","precision","recall","f1_score"]
#dfMetricsClass = dsMetrics.loc[dsMetrics['class'] == "asleep"]
ss = dataMetrics.loc[dataMetrics['class'] == "avg"]

ss.loc[ss['accuracy'] == ss['accuracy'].max()][metrics]

,class,epochs,accuracy,precision,recall,f1_score
341,avg,1,0.82507,0.746244,0.555368,0.55626


In [ ]:
ss.loc[ss['recall'] == ss['recall'].max()][metrics]